In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import zipfile
import glob
from numpy import save
import os
import skimage
from skimage.transform import resize
import pickle, os, sqlite3, random

In [2]:
IMG_SIZE=80
CLASS_NAMES = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','nothing','space']

In [ ]:
train_len=87000

In [16]:
train_filepath='C:/Users/Shakshi/Downloads/sign_train/asl_alphabet_train/A1.jpg'

In [1]:
img=cv2.imread('C:/Users/Shakshi/Downloads/sign_train/asl_alphabet_train/A')
hist = get_hand_hist()
x, y, w, h = 300, 100, 300, 300
img = cv2.flip(img, 1)
imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
dst = cv2.calcBackProject([imgHSV], [0, 1], hist, [0, 180, 0, 256], 1)
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
cv2.filter2D(dst,-1,disc,dst)
blur = cv2.GaussianBlur(dst, (11,11), 0)
blur = cv2.medianBlur(blur, 15)
thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
thresh = cv2.merge((thresh,thresh,thresh))
thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
thresh = thresh[y:y+h, x:x+w]
contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

if len(contours) > 0:
    contour = max(contours, key = cv2.contourArea)
    if cv2.contourArea(contour) > 10000 and frames > 50 :
        x1, y1, w1, h1 = cv2.boundingRect(contour)
        save_img = thresh[y1:y1+h1, x1:x1+w1]
        if w1 > h1:
            save_img = cv2.copyMakeBorder(save_img, int((w1-h1)/2) , int((w1-h1)/2) , 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
        elif h1 > w1:
            save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1-w1)/2) , int((h1-w1)/2) , cv2.BORDER_CONSTANT, (0, 0, 0))
        save_img = cv2.resize(save_img, (image_x, image_y))
        rand = random.randint(0, 10)
        if rand % 2 == 0:
            save_img = cv2.flip(save_img, 1)
        cv2.putText(img, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.imshow("Capturing gesture", img)
        cv2.imshow('thresh ',thresh);
        cv2.imshow('save image',save_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [33]:
X_train = np.empty((train_len, IMG_SIZE, IMG_SIZE, 1), dtype=np.float32)
y_train=np.empty((train_len,), dtype=np.int)

In [34]:
i=0
# minValue=70
for folder_name in os.listdir(train_filepath):
    for image_path in os.listdir(train_filepath+folder_name):
        img=cv2.imread(train_filepath+folder_name+'/'+image_path,cv2.IMREAD_GRAYSCALE)
        if(img is not None):
            blur = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
            th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
            ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            res=resize(res,(IMG_SIZE,IMG_SIZE,1))
            img_arr=np.asarray(res).reshape((-1,IMG_SIZE,IMG_SIZE,1))
            X_train[i]=img_arr
            y_train[i]=CLASS_NAMES.index(folder_name)
            i=i+1

In [35]:
# save('X_train_sr.npy',X_train)

In [36]:
# save('y_train_sr.npy',y_train)

In [3]:
test_filepath='C:/Users/Shakshi/Downloads/sign_test/asl_alphabet_test/'

In [4]:
test_len=28

In [5]:
X_test=np.empty((test_len,IMG_SIZE,IMG_SIZE,1),dtype=np.float32)
y_test=np.empty((test_len,),dtype=np.int8)

In [6]:
i=0
for image_path in os.listdir(test_filepath):
    img=cv2.imread(test_filepath+'/'+image_path,cv2.IMREAD_GRAYSCALE)
    if(img is not None):
        blur = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
        th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
        ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        res=resize(res,(IMG_SIZE,IMG_SIZE,1))
        img_arr=np.asarray(res).reshape((-1,IMG_SIZE,IMG_SIZE,1))
#         if(i==0):
#             cv2.imshow('image',res)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
        X_test[i]=img_arr
        y_test[i]=CLASS_NAMES.index(image_path.split('_')[0])
        i=i+1

In [7]:
# save('X_test_sr.npy',X_test)
# save('y_test_sr.npy',y_test)

In [8]:
# X_train.shape

In [9]:
cv2.imshow('image',X_test[9])

cv2.waitKey(0)
cv2.destroyAllWindows()